In [1]:
import atd2022
import torch
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sktime.performance_metrics.forecasting import (
    MeanAbsoluteScaledError,
    MeanSquaredError,
)
import my_mod as util
from models import CnnForecaster

In [2]:
import torch
torch.cuda.is_available()

True

# Small Scale Test for GPU support

In [3]:
truth = atd2022.io.read_csv()

In [4]:
test = truth[["AA", "AC"]]

In [5]:
lispCNN = CnnForecaster(training_epochs = 2)

In [6]:
lispCNN.fit(data=test)

0
epochs 1/2


/scratch/wzong/anaconda3/envs/atd2022/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1, 20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


train_loss 105.81926727294922
epochs 2/2
train_loss 103.5216293334961
1
epochs 1/2
train_loss 488.1627197265625
epochs 2/2
train_loss 476.9712219238281


In [7]:
res=lispCNN.predict(test.tail(4).index)

In [8]:
res

AA                                                    \
                        1    2    3    4    5    6    7    8    9    10   11   
2018-01-15/2018-01-21  0.0 -0.0  1.0 -0.0  0.0  0.0 -0.0  0.0 -0.0  1.0  1.0   
2018-01-22/2018-01-28  0.0  0.0  0.0  0.0 -0.0  1.0  0.0  0.0 -0.0  1.0  0.0   
2018-01-29/2018-02-04  0.0 -0.0 -0.0  0.0  0.0  0.0  0.0 -0.0  0.0  0.0  0.0   
2018-02-05/2018-02-11  0.0 -0.0  0.0  0.0  0.0  0.0  0.0 -0.0  0.0  0.0 -0.0   

                                                                     AC       \
                        12   13   14   15   16   17   18   19   20   1    2    
2018-01-15/2018-01-21  1.0  0.0  0.0  0.0 -0.0  2.0  0.0  0.0  0.0  0.0  1.0   
2018-01-22/2018-01-28  1.0  0.0 -0.0 -0.0 -0.0  1.0  1.0 -0.0  0.0 -0.0  0.0   
2018-01-29/2018-02-04  0.0  0.0  0.0  0.0  0.0  0.0 -0.0 -0.0  0.0  0.0 -0.0   
2018-02-05/2018-02-11  0.0  0.0 -0.0  0.0  0.0  0.0 -0.0 -0.0 -0.0  0.0 -0.0   

                                                                              \
                        3    4    5    6    7    8    9    10   11   12   13   
2018-01-15/2018-01-21  2.0  1.0  1.0  0.0  0.0  1.0 -0.0  1.0  1.0  1.0 -0.0   
2018-01-22/2018-01-28  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0   
2018-01-29/2018-02-04  0.0  0.0  0.0  0.0  0.0 -0.0  0.0  0.0  0.0  0.0 -0.0   
2018-02-05/2018-02-11 -0.0  0.0  0.0  0.0  0.0 -0.0  0.0  0.0 -0.0  0.0  0.0   

                                                          
                        14   15   16   17   18   19   20  
2018-01-15/2018-01-21  0.0  0.0  0.0  3.0 -0.0  0.0  1.0  
2018-01-22/2018-01-28  1.0  0.0 -0.0  2.0  1.0 -0.0  0.0  
2018-01-29/2018-02-04  0.0  0.0  0.0  0.0 -0.0 -0.0  0.0  
2018-02-05/2018-02-11  0.0 -0.0  0.0  0.0 -0.0 -0.0 -0.0

# Full Scale Test

In [9]:
lispCNN = CnnForecaster(training_epochs = 1)

In [10]:
 # Load data
truth = atd2022.io.read_csv()

# Subset the data for the sake of making this fast.
# Remove if you want to run the example on the entire dataset.
truth = truth.head(80)

# Experiment Parameters
window = 100
num_predict = 4
gap = 0
slide = 1

# Create a dataset `Splitter` object for generating train/test splits
splitter = atd2022.backtest.Splitter(
    truth,
    window,
    num_predict,
    gap,
    slide,
    expanding=True,
) 

In [12]:
 # Populate a list of models that support the `atd2022.forecasters.Forecaster` protocol
# with which will we generate historical forecasts
models = [
    lispCNN,
    atd2022.forecasters.PredictMeanForecaster(),
    atd2022.forecasters.ExponentiallyWeightedMovingAverage(),
]

# Compute historical forecasts for all models
predictions = [
    atd2022.backtest.historical_forecast(model, splitter, verbose=True)
    for model in models
]

ValueError: Historical forecasts require a splitter of nonzero length.

In [ ]:
 metric_functions = [
    MeanAbsoluteScaledError(),
    MeanSquaredError(square_root=True),
]

metrics_df = atd2022.metrics.compute_metrics(
    truth,
    predictions,
    metric_functions,
    models=models,
    train=truth,
    verbose=True,
)

metrics_n = atd2022.metrics.compute_nstep_metrics(
    truth,
    predictions,
    metric_functions,
    models=models,
    train=truth,
    verbose=True,
)
display(metrics_df)
display(metrics_n)
display(atd2022.viz.plot_nstep_metrics(metrics_n))